In [5]:
from Models.PPO.PPO_Agent import PPO_Agent
import torch
import slimevolleygym
from slimevolleygym import BaselinePolicy
from utils import convert_to_vector, convert_to_value
import types
import slimevolleygym
import slimevolleygym.mlp as mlp
from slimevolleygym.mlp import Model
import json

In [6]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

# Print the device as a check
print("Device used: ", DEVICE)

Device used:  cuda:0


In [7]:
env = slimevolleygym.SlimeVolleyEnv()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#
# Pick one of the baselines below to use as the first agent
#

# 1- PPO trained against the baseline
# agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[256, 128])
# agent1.load_models("Logging/PPO-BASELINE/20240408-190946-lr-0.001-entcoef-0-mlp-256-128-kl-0.02", 1, 12337140)

# 2- Genetic agent
agent1 = Model(mlp.games['slimevolleylite'])
with open('Logging/GENETIC-SELFPLAY/20240409-021844-numagents-128-totalnumgames-546000/game_546000') as f:
    d = json.load(f)
    agent1.set_model_params(d[0])
def select_action(self, state, greedy=False):
    action = self.predict(state, mean_mode=greedy)
    action = (action > 0).astype(int) # Anything positive means a 1, 0 or negative means a 0
    return convert_to_value(action), None
def evaluation_mode(self):
    pass
agent1.select_action = types.MethodType(select_action, agent1)
agent1.evaluation_mode = types.MethodType(evaluation_mode, agent1)

#
# Pick one of the snippets below to use as the second agent
#

# 1- Baseline
agent2 = BaselinePolicy()
def select_action(self, state, greedy=False):
    action = self.predict(state)
    return convert_to_value(action), None
def evaluation_mode(self):
    pass
agent2.select_action = types.MethodType(select_action, agent2)
agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# # 2- Random agent
# def select_action(self, state, greedy=False):
#     action = convert_to_value(env.action_space.sample())
#     return convert_to_vector(action), None
# def evaluation_mode(self):
#     pass
# agent2.select_action = types.MethodType(select_action, agent2)
# agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# # 3- Genetic agent
# agent2 = Model(mlp.games['slimevolleylite'])
# with open('Logging/GENETIC-SELFPLAY/20240409-021844-numagents-128-totalnumgames-546000/game_546000') as f:
#     d = json.load(f)
#     agent2.set_model_params(d[0])

C:\Users\wassi\AppData\Local\Temp\ipykernel_20012\2741364281.py:14: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  agent1 = Model(mlp.games['slimevolleylite'])
C:\Users\wassi\AppData\Local\Temp\ipykernel_20012\2741364281.py:17: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  agent1.set_model_params(d[0])


In [8]:
# Set the agents to evaluation mode
agent1.evaluation_mode()
agent2.evaluation_mode()

# Run num_eval_episodes episodes and calculate the total return
total_return = 0
for _ in range(10):

    state1 = env.reset()
    state2 = state1
    done = False
    while not done:

        # Render the environment
        env.render(mode="human")
        
        with torch.no_grad():

            # Select the actions for each agent
            action1, _ = agent1.select_action(state1, greedy=True)
            action2, _ = agent2.select_action(state2, greedy=True)
        
        # Step the environment forward
        next_state1, reward, done, info = env.step(convert_to_vector(action1), otherAction=convert_to_vector(action2))
        next_state2 = info['otherObs']
        
        # Add the individual agents' rewards to the total returns (Since they're the same for both agents)
        total_return += reward

        # Update the states
        state1 = next_state1
        state2 = next_state2